# Phoenix-Evals 2.0: Preview


In [ ]:
! pip install arize-phoenix arize-phoenix-evals==0.29.0 openai pandas openinference-instrumentation-openai

I0000 00:00:1756180440.740726 1910735 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


In [26]:
import phoenix as px
from phoenix.otel import register

px.launch_app()
tracer_provider = register(auto_instrument=True)

Existing running Phoenix instance detected! Shutting it down and starting a new instance...
⚠️ PHOENIX_COLLECTOR_ENDPOINT is set to https://app.phoenix.arize.com/s/ehutton.
⚠️ This means that traces will be sent to the collector endpoint and not this app.
⚠️ If you would like to use this app to view traces, please unset this environmentvariable via e.g. `del os.environ['PHOENIX_COLLECTOR_ENDPOINT']` 
⚠️ You will need to restart your notebook to apply this change.
/Users/elizabethhutton/Projects/phoenix/src/phoenix/otel/otel.py:333: UserWarning: Could not infer collector endpoint protocol, defaulting to HTTP.
  warnings.warn("Could not infer collector endpoint protocol, defaulting to HTTP.")
Overriding of current TracerProvider is not allowed


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/s/ehutton/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



/Users/elizabethhutton/Projects/phoenix/src/phoenix/otel/otel.py:596: UserWarning: No OpenInference instrumentors found. Maybe you need to update your OpenInference version? Skipping auto-instrumentation.
  warnings.warn(


In [26]:
import phoenix as px
from phoenix.otel import register

px.launch_app()
tracer_provider = register(auto_instrument=True)

Existing running Phoenix instance detected! Shutting it down and starting a new instance...
⚠️ PHOENIX_COLLECTOR_ENDPOINT is set to https://app.phoenix.arize.com/s/ehutton.
⚠️ This means that traces will be sent to the collector endpoint and not this app.
⚠️ If you would like to use this app to view traces, please unset this environmentvariable via e.g. `del os.environ['PHOENIX_COLLECTOR_ENDPOINT']` 
⚠️ You will need to restart your notebook to apply this change.
/Users/elizabethhutton/Projects/phoenix/src/phoenix/otel/otel.py:333: UserWarning: Could not infer collector endpoint protocol, defaulting to HTTP.
  warnings.warn("Could not infer collector endpoint protocol, defaulting to HTTP.")
Overriding of current TracerProvider is not allowed


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/s/ehutton/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



/Users/elizabethhutton/Projects/phoenix/src/phoenix/otel/otel.py:596: UserWarning: No OpenInference instrumentors found. Maybe you need to update your OpenInference version? Skipping auto-instrumentation.
  warnings.warn(


## LLM Configuration

**Core Design Principle:** The library should work with any LLM model and provider.

The LLM wrapper unifies generation tasks across model providers by delegating to the most commonly installed client SDKs (OpenAI, LangChain, LiteLLM) via adapters.


In [27]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [28]:
from phoenix.evals.preview.llm import LLM, show_provider_availability

show_provider_availability()  # shows which providers/clients are available based on what's installed in your environment


📦 AVAILABLE PROVIDERS (sorted by client priority)
--------------------------------------------------------------------
Provider  | Status      | Client    | Dependencies                  
--------------------------------------------------------------------
openai    | ✓ Available | openai    | openai               
openai    | ✓ Available | langchain | langchain, langchain_openai
openai    | ✓ Available | litellm   | litellm              
anthropic | ✓ Available | langchain | langchain, langchain_anthropic
anthropic | ✓ Available | litellm   | litellm              


In [29]:
llm = LLM(
    provider="openai", model="gpt-4o"
)  # you could also specify the client e.g. "langchain" or "openai"

### Prompt Templates

The new `Template` primitive provides a unified interface for both f-string and mustache formatted string prompt templates.


In [30]:
from phoenix.evals.preview.templating import Template

prompt_template = Template(template="Classify this message as spam or not: {message}")
print(prompt_template.variables)
print(prompt_template.render({"message": "Hello, world!"}))

['message']
Classify this message as spam or not: Hello, world!


## About the `Score` Data Model

All evaluators output a list of `Score` objects with some or all of the following properties:

**Core Design Principle:** The output of evaluators should be rich with information.

- **name**: the name of the score
- **score**: numeric score
- **label**: str label for categorical evals
- **explanation**: an explanation for the result
- **direction**: optimization direction, either maximize or minimize
- **source**: source of the eval (llm, heuristic, or human)
- **metadata**: other metadata attached to the score

**Note:** evaluations always return a **list** of `Score` objects. Often, this will be a list of length 1, but some evaluators may return multiple scores for a single `eval_input` (e.g. precision/recall or multi-criteria evals).


## Built-In Metrics


### Hallucination (LLM classification)


In [5]:
from phoenix.evals.preview.metrics import (
    HallucinationEvaluator,
    PrecisionRecallFScore,
    exact_match,
)

llm = LLM(provider="openai", model="gpt-4o-mini")
hallucination_evaluator = HallucinationEvaluator(llm=llm)
result = hallucination_evaluator(
    {
        "input": "What is the capital of France?",
        "output": "Paris is the capital of France.",
        "context": "Paris is the capital and largest city of France.",
    }
)
print("Hallucination result:")
result[0].pretty_print()

Hallucination result:
{
  "name": "hallucination",
  "score": 1.0,
  "label": "factual",
  "explanation": "The response correctly states that Paris is the capital of France, which aligns perfectly with the provided context.",
  "metadata": {
    "model": "gpt-4o-mini"
  },
  "source": "llm",
  "direction": "maximize"
}


### Exact Match (heuristic)


In [6]:
result = exact_match({"output": "no", "expected": "yes"})
print("Exact match result:")
result[0].pretty_print()

Exact match result:
{
  "name": "exact_match",
  "score": 0.0,
  "metadata": {},
  "source": "heuristic",
  "direction": "maximize"
}


### Precision, Recall, F1 (multi-score)

A single evaluator can return multiple scores!

Notes:

- Works for binary or multi-class labels, as well as integer values.
- Provide positive label for best results. If binary, 1.0 is presumed positive.
- Default F score is F1, but beta is configurable.
- Default averaging technique is macro, but it is configurable


In [7]:
precision_recall_fscore = PrecisionRecallFScore(positive_label="yes")
result = precision_recall_fscore({"output": ["no", "yes", "yes"], "expected": ["yes", "no", "yes"]})
print("Results:")
print(result[0])
print(result[1])
print(result[2])

Results:
Score(name='precision', score=0.5, label=None, explanation=None, metadata={'beta': 1.0, 'average': 'macro', 'labels': ['yes', 'no'], 'positive_label': 'yes'}, source='heuristic', direction='maximize')
Score(name='recall', score=0.5, label=None, explanation=None, metadata={'beta': 1.0, 'average': 'macro', 'labels': ['yes', 'no'], 'positive_label': 'yes'}, source='heuristic', direction='maximize')
Score(name='f1', score=0.5, label=None, explanation=None, metadata={'beta': 1.0, 'average': 'macro', 'labels': ['yes', 'no'], 'positive_label': 'yes'}, source='heuristic', direction='maximize')


## Input Mapping and Transformation

**Core Design Principle:** The inputs to an evaluator should be well-defined and discoverable.

Every evaluator has an `input_schema` which describes what inputs it expects.


### Use `.describe()` to inspect an `Evaluator`'s input schema

Because pydantic `BaseModel` is used for the `input_schema`, input fields can be annotated with types, descriptions, and even aliases.


In [8]:
# describe an evaluator to inspect its input schema
hallucination_evaluator.describe()  # requires strings for input, output, and context

{'name': 'hallucination',
 'source': 'llm',
 'direction': 'maximize',
 'input_schema': {'properties': {'input': {'description': 'The input query.',
    'title': 'Input',
    'type': 'string'},
   'output': {'description': 'The response to the query.',
    'title': 'Output',
    'type': 'string'},
   'context': {'description': 'The context or reference text.',
    'title': 'Context',
    'type': 'string'}},
  'required': ['input', 'output', 'context'],
  'title': 'HallucinationInputSchema',
  'type': 'object'}}

In [9]:
exact_match.describe()  # requires string output and expected

{'name': 'exact_match',
 'source': 'heuristic',
 'direction': 'maximize',
 'input_schema': {'properties': {'output': {'title': 'Output',
    'type': 'string'},
   'expected': {'title': 'Expected', 'type': 'string'}},
  'required': ['output', 'expected'],
  'title': 'Exact_matchInput',
  'type': 'object'}}

### Use `input_mapping` to map/transform data into expected `input_schema`

**Core Design Principle:** You should not have to modify your data to run evaluations.

An evaluator's input arguments may not perfectly match those in your example or dataset. Or, you may want to run multiple evaluators on the same example, but they have different or conflicting `input_schema`'s.

To extract the values from a nested `eval_input` payload, provide an `input_mapping` that maps evaluator's input fields to a path spec in your original data.

Possible Mapping Values:

- top-level keys in your
- a path spec following glom syntax
- callable functions


In [10]:
# example nested eval input for a RAG system
eval_input = {
    "input": {"query": "user input query"},
    "output": {
        "responses": ["model answer", "model answer 2"],
        "documents": ["doc A", "doc B"],
    },
    "expected": "correct answer",
}

# in order to run the hallucination evaluator, we need to process the eval_input to the fit the input schema
input_mapping = {
    "input": "input.query",  # dot notation to access nested keys
    "output": "output.responses.0",  # dot notation to access list indices
    "context": lambda x: " ".join(
        x["output"]["documents"]
    ),  # lambda function to combine the document chunks
}

# the evaluator uses the input_mapping to transform the eval_input into the expected input schema
result = hallucination_evaluator.evaluate(eval_input, input_mapping)
result[0].pretty_print()

{
  "name": "hallucination",
  "score": 0.0,
  "label": "hallucinated",
  "explanation": "The response cannot be determined without specific details about the query, context, and the response itself. The placeholders do not provide enough information to classify as factual or hallucinated.",
  "metadata": {
    "model": "gpt-4o-mini"
  },
  "source": "llm",
  "direction": "maximize"
}


### Use `bind_evaluator` to bind an `input_mapping` to an `Evaluator` for reuse

Note: We don't need to remap "expected" for the `exact_match` eval because it already exists in our `eval_input`


In [11]:
from phoenix.evals.preview import bind_evaluator

# we can bind an input_mapping to an evaluator ahead of call time for easier sequential evals
evaluators = [
    bind_evaluator(hallucination_evaluator, input_mapping),
    bind_evaluator(exact_match, {"output": "output.responses.0"}),
]
scores = []
for evaluator in evaluators:
    scores.append(evaluator.evaluate(eval_input))  # no need to pass input_mapping each time

[score[0].pretty_print() for score in scores]

/var/folders/w6/w8lrsrb50r545rdslc0w9fpr0000gn/T/ipykernel_20624/933383315.py:1: UserWarning: 

⚠️ EXPERIMENTAL: The phoenix.evals.preview module and all its components are experimental and subject to change without notice. This code should not be used in production.
  from phoenix.evals.preview import bind_evaluator


{
  "name": "hallucination",
  "score": 0.0,
  "label": "hallucinated",
  "explanation": "The context provided is insufficient to determine the accuracy of the response, as it does not contain any specific details or content to analyze. Therefore, I cannot confirm whether the response is factual or not.",
  "metadata": {
    "model": "gpt-4o-mini"
  },
  "source": "llm",
  "direction": "maximize"
}
{
  "name": "exact_match",
  "score": 0.0,
  "metadata": {},
  "source": "heuristic",
  "direction": "maximize"
}


[None, None]

## About the `Evaluator` Abstraction

- sync and async methods for single record evals
- evaluators are directly callable e.g. `evaluator(eval_input)` in addition to `evaluator.evaluate(eval_input)`
- inheritors of the base class only have to implement `_evaluate` and the remaining methods come for free unless explicitly overwritten
- all evaluators have a well-defined `input_schema` that, if not provided at instantiation, is inferred from either the prompt template (for LLM evaluators) or decorated function signature (for heuristic evaluators)
- accept an arbitrary `eval_input` payload, and an optional `input_mapping` to map/transform the `eval_input` to match the `input_schema`. Input remapping is handled by the base `Evaluator` class.
- evaluations always return a **list** of `Score` objects. Often, this will be a list of length 1, but some evaluators may return multiple scores for a single `eval_input` (e.g. precision/recall or multi-criteria evals).

## About the `ClassificationEvaluator`

- designed to replace `llm_classify` functionality
- requires an LLM, prompt template, and label choices
- allows user to specify labels and custom score mapping via `choices`, with various supported formats
- option to turn off explanations, though they are included by default in accordance with best practices
- requires the LLM to have some kind of tool calling or structured output ability


## Custom LLM Classification Evaluators

This is similar to `llm_classify`, for LLM-as-a-judge evaluations that output a label and explanation.

**New features:**

- specify scores for each label
- run on single records (not just a dataframe)
- leverages model tool calling / structured output for more reliable output parsing

**Extensions:**

This abstraction can be easily extended to support multi-criteria evaluations where a judge is asked to evaluate an input across multiple dimensions in one request.


You can specify classification labels/scores in 3 ways:

1. String labels only:
   ```python
   "positive", "negative", "neutral"]
   ```
2. Labels mapped to scores (**recommended**):
   ```python
   {"positive": 1.0, "negative": 0.0, "neutral": 0.5}
   ```
3. Labels mapped to scores + descriptions\*:
   ```python
   {"positive": (1.0, "Expresses happiness or satisfaction"),
   "negative": (0.0, "Expresses disappointment or anger"),
   "neutral": (0.5, "Expresses neither positive nor negative feelings")}
   ```
   \*Note: tool calling consistency with this schema is not as reliable or consistent across models


In [13]:
from phoenix.evals.preview import ClassificationEvaluator, create_classifier

evaluator = ClassificationEvaluator(
    name="sentiment",
    llm=llm,
    prompt_template="Classify the sentiment of this text: {text}",
    choices={"positive": 1.0, "negative": 0.0, "neutral": 0.5},
)

result = evaluator.evaluate({"text": "I love this!"})
result[0].pretty_print()

{
  "name": "sentiment",
  "score": 1.0,
  "label": "positive",
  "explanation": "The text \"I love this!\" expresses a strong positive emotion of love or enthusiasm, indicating a favorable sentiment.",
  "metadata": {
    "model": "gpt-4o-mini"
  },
  "source": "llm",
  "direction": "maximize"
}


Classification evaluators can also be created using a factory function:


In [14]:
evaluator = create_classifier(
    name="spam_classifier",
    llm=llm,
    prompt_template="Classify this message as spam or not: {message}",
    choices={"spam": 0.0, "not_spam": 1.0},
)
evaluator.evaluate({"message": "Buy now! Limited time offer!"})

[Score(name='spam_classifier', score=0.0, label='spam', explanation="The message uses urgent language often associated with spam, such as 'buy now' and 'limited time offer'. These phrases are typically designed to create a sense of urgency to pressure recipients into making a purchase, which is a common trait of spam messages.", metadata={'model': 'gpt-4o-mini'}, source='llm', direction='maximize')]

For more complex LLM evaluation tasks that don't fit the classification mold, there is an `LLMEvaluator` class with open-ended tool calling support that can be inherited.


## About the `create_evaluator` decorator


In [15]:
from phoenix.evals.preview import create_evaluator


# heuristic evaluator that returns a tuple of score and label
@create_evaluator(name="text_length")
def text_length_score(text: str) -> tuple[float, str]:
    """Score text based on length (longer = better, up to a point)"""
    length = len(text)
    if length < 10:
        score = 0.0
        label = "too_short"
    elif length < 50:
        score = 0.5
        label = "short"
    elif length < 200:
        score = 1.0
        label = "good_length"
    else:
        score = 0.8
        label = "too_long"

    return (score, label)


text_length_score(eval_input={"text": "This is a test"})

[Score(name='text_length', score=0.5, label='short', explanation=None, metadata={}, source='heuristic', direction='maximize')]

In [17]:
from phoenix.evals.preview import Score, create_evaluator


# heuristic evaluator that returns a Score object with metadata
@create_evaluator(name="keyword_presence")
def keyword_presence_score(text: str, keywords: list[str]) -> tuple[float, str, str]:
    """Score text based on presence of keywords"""
    text_lower = text.lower()
    keyword_list = keywords

    found_keywords = [k for k in keyword_list if k in text_lower]
    score = len(found_keywords) / len(keyword_list) if keyword_list else 0.0

    return Score(
        score=score,
        label=f"found_{len(found_keywords)}_of_{len(keyword_list)}",
        explanation=f"Found keywords: {found_keywords}",
        metadata={"found_keywords": found_keywords, "total_keywords": len(keyword_list)},
    )


keyword_presence_score.describe()  # input schema is inferred from the function signature

{'name': 'keyword_presence',
 'source': 'heuristic',
 'direction': 'maximize',
 'input_schema': {'properties': {'text': {'title': 'Text', 'type': 'string'},
   'keywords': {'items': {'type': 'string'},
    'title': 'Keywords',
    'type': 'array'}},
  'required': ['text', 'keywords'],
  'title': 'Keyword_presenceInput',
  'type': 'object'}}

## Dataframe Evaluation

Run multiple evaluators over a pandas dataframe. The output is an augmented dataframe with two added columns per score:

1. `{score_name}_score` contains the JSON serialized score (or None if the evaluation failed)
2. `{evaluator_name}_execution_details` contains information about the execution status, duration, and any exceptions that ocurred.

Notes:

- use `bind_evaluator` to bind `input_mappings` to your evaluators so they match your dataframe columns.


In [1]:
import pandas as pd

from phoenix.evals.preview.evaluators import evaluate_dataframe
from phoenix.evals.preview.metrics import PrecisionRecallFScore

precision_recall_fscore = PrecisionRecallFScore(positive_label="Yes")

df = pd.DataFrame(
    {
        "output": [["Yes", "Yes", "No"], ["Yes", "No", "No"]],
        "expected": [["Yes", "No", "No"], ["Yes", "No", "No"]],
    }
)

result = evaluate_dataframe(df, [precision_recall_fscore])
result.head()

/Users/elizabethhutton/Projects/phoenix/.venv/lib/python3.9/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


,output,expected,precision_recall_fscore_execution_details,precision_score,recall_score,f1_score
0,"[Yes, Yes, No]","[Yes, No, No]","{""status"": ""COMPLETED"", ""exceptions"": [], ""exe...","{""name"": ""precision"", ""score"": 0.5, ""metadata""...","{""name"": ""recall"", ""score"": 1.0, ""metadata"": {...","{""name"": ""f1"", ""score"": 0.6666666666666666, ""m..."
1,"[Yes, No, No]","[Yes, No, No]","{""status"": ""COMPLETED"", ""exceptions"": [], ""exe...","{""name"": ""precision"", ""score"": 1.0, ""metadata""...","{""name"": ""recall"", ""score"": 1.0, ""metadata"": {...","{""name"": ""f1"", ""score"": 1.0, ""metadata"": {""bet..."


In [2]:
from phoenix.evals.preview.evaluators import bind_evaluator
from phoenix.evals.preview.llm import LLM
from phoenix.evals.preview.metrics import HallucinationEvaluator, exact_match

df = pd.DataFrame(
    {
        "output": ["Yes", "Yes", "No"],
        "expected": ["Yes", "No", "No"],
        "context": ["This is a test", "This is another test", "This is a third test"],
        "query": [
            "What is the name of this test?",
            "What is the name of this test?",
            "What is the name of this test?",
        ],
        "response": ["First test", "Another test", "Third test"],
    }
)

llm = LLM(provider="openai", model="gpt-4o")

hallucination_evaluator = bind_evaluator(
    HallucinationEvaluator(llm=llm), {"input": "query", "output": "response"}
)

result = evaluate_dataframe(df, [exact_match, hallucination_evaluator])
result.head()

,output,expected,context,query,response,exact_match_execution_details,hallucination_execution_details,exact_match_score,hallucination_score
0,Yes,Yes,This is a test,What is the name of this test?,First test,"{""status"": ""COMPLETED"", ""exceptions"": [], ""exe...","{""status"": ""COMPLETED"", ""exceptions"": [], ""exe...","{""name"": ""exact_match"", ""score"": 1.0, ""metadat...","{""name"": ""hallucination"", ""score"": 0.0, ""label..."
1,Yes,No,This is another test,What is the name of this test?,Another test,"{""status"": ""COMPLETED"", ""exceptions"": [], ""exe...","{""status"": ""COMPLETED"", ""exceptions"": [], ""exe...","{""name"": ""exact_match"", ""score"": 0.0, ""metadat...","{""name"": ""hallucination"", ""score"": 1.0, ""label..."
2,No,No,This is a third test,What is the name of this test?,Third test,"{""status"": ""COMPLETED"", ""exceptions"": [], ""exe...","{""status"": ""COMPLETED"", ""exceptions"": [], ""exe...","{""name"": ""exact_match"", ""score"": 1.0, ""metadat...","{""name"": ""hallucination"", ""score"": 1.0, ""label..."


# Practice: BYO Judge

**Your task:** Create a custom LLM judge to classify text complexity. Inputs can be classified into one of the following labels: simple, moderate, or complex. For your use case, simple text is better than moderate or complex.

Use the following 3 examples to test your new evaluator:


In [ ]:
data = [
    {
        "text": "AI is when computers learn to do things like people, like recognizing faces or playing games."
    },
    {
        "text": "Machine learning is a method in artificial intelligence where systems improve their performance by learning from data, without being explicitly programmed for each task"
    },
    {
        "text": "Artificial intelligence systems employing deep reinforcement learning utilize hierarchical neural architectures to iteratively optimize policy gradients across high-dimensional state-action spaces, converging toward sub-optimal equilibria in stochastic environments via backpropagated reward signals and temporally extended credit assignment mechanisms."
    },
]

In [ ]:
# write your judge here

In [12]:
# test your judge on the examples here

# Practice: DIY Heuristic Evaluator

**Your task:** Turn the following function into an Evaluator that calculates the Levenshtein distance between two strings.

Note: Smaller values indicate higher similarity (lower score = better).

Run the Evaluator on the following data:


In [ ]:
eval_input = {
    "input": {"query": "What is the capital of France?"},
    "output": {"response": "It is Paris"},
    "expected": "Paris",
}

In [24]:
# turn this function into a heuristic evaluator
def levenshtein_distance(s1: str, s2: str) -> int:
    """
    Compute the Levenshtein distance between two strings s1 and s2.
    """
    m, n = len(s1), len(s2)

    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            cost = 0 if s1[i - 1] == s2[j - 1] else 1
            dp[i][j] = min(dp[i - 1][j] + 1, dp[i][j - 1] + 1, dp[i - 1][j - 1] + cost)

    return dp[m][n]

In [ ]:
# test your evaluator on the example above.
# hint: use an input_mapping to map/transform the input to the function's expected arguments.